In [ ]:
import cv2
import numpy as np

# Load the grayscale image
image = cv2.imread('/content/lena_gray.png', cv2.IMREAD_GRAYSCALE).astype(np.float32)

# Add random noise with mean 0 and standard deviation 20
noise = np.random.normal(0, 20, image.shape)#.astype(np.float32)
noisy_image = (image + noise)#.astype(np.float32)
cv2.imwrite('noisy_image.jpg', noisy_image)
image2 = cv2.imread('/content/noisy_image.jpg', cv2.IMREAD_GRAYSCALE).astype(np.float32)

# Apply average filter with kernel size 7x7
kernel_size = (15, 15)
filtered_image = cv2.blur(noisy_image, kernel_size)

# Save the results
cv2.imwrite('_image.jpg', image2-noise)
# cv2.imwrite('original_image.jpg', np.clip(noisy_image - filtered_image, 0, 255))
# cv2.imwrite('image.jpg', np.clip(noisy_image - image, 0, 255))
print(image2-noise,image)
import numpy as np
from skimage.metrics import peak_signal_noise_ratio

def calculate_psnr(original_image, reconstructed_image):
    # Assuming both images are in the range [0, 255]
    original_image = np.asarray(original_image, dtype=np.float64)
    reconstructed_image = np.asarray(reconstructed_image, dtype=np.float64)

    # Calculate PSNR using scikit-image library
    psnr_value = peak_signal_noise_ratio(original_image, reconstructed_image, data_range=255)

    return psnr_value

# Example usage:
# original_image and reconstructed_image should be NumPy arrays representing the images
psnr_result = calculate_psnr(image,image2-noise)
print(f"PSNR: {psnr_result} dB")



[[59.77777  60.       60.555557 ... 64.888885 63.555557 43.33333 ]
 [60.66667  59.888885 59.444443 ... 63.888885 63.555557 43.33333 ]
 [59.33333  60.77777  60.444443 ... 63.77777  63.444443 43.111115]
 ...
 [59.88889  57.       61.444443 ... 61.333336 57.888885 59.444443]
 [59.555557 57.88889  64.       ... 61.111115 61.555557 63.555557]
 [60.444443 56.555557 64.33333  ... 60.111115 59.       62.444443]]


In [ ]:
import cv2
import numpy as np

# Function to generate Gaussian noise
def generate_gaussian_noise(mean, stddev, size):
    return np.random.normal(mean, stddev, size)

# Load the grayscale image
image = cv2.imread('/content/lena_gray.png', cv2.IMREAD_GRAYSCALE).astype(np.float32)

# Set mean and standard deviation for Gaussian noise
mean = 0
stddev = 20
size = image.shape

# Generate initial Gaussian noise
initial_noise = generate_gaussian_noise(mean, stddev, size).astype(np.float32)

# Rotate the noise 4 times (90 degrees each time) and create mirror images
rotated_noises = [np.rot90(initial_noise, k=i) for i in range(4)]
mirrored_rotated_noises = [np.fliplr(noise) for noise in rotated_noises]

# Combine all 8 noises
all_noises = [initial_noise] + rotated_noises + mirrored_rotated_noises

# Calculate the average of the noises
average_noise = np.mean(all_noises, axis=0)

# Add the average noise to the original image
noisy_image = (image + average_noise).astype(np.float32)

# Display the images
cv2.imwrite('Original Image.png',(image + initial_noise).astype(np.float32))
cv2.imwrite('Noisy Image.png', noisy_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


True

In [ ]:
import cv2
import numpy as np

def calculate_psnr(original_image, reconstructed_image):
    # Ensure images have the same size and format
    assert original_image.shape == reconstructed_image.shape, "Images must have the same dimensions"
    assert original_image.dtype == reconstructed_image.dtype, "Images must have the same data type"

    # Calculate Mean Squared Error (MSE)
    mse = np.mean((original_image - reconstructed_image) ** 2)

    # Calculate PSNR
    max_pixel_value = 255  # Assuming 8-bit images
    psnr = 10 * np.log10((max_pixel_value ** 2) / mse)

    return psnr

# Load your original and reconstructed images
original_image = cv2.imread("/content/lena_gray.png", cv2.IMREAD_GRAYSCALE)
reconstructed_image = cv2.imread("/content/Noisy Image.png", cv2.IMREAD_GRAYSCALE)

# Convert images to float for more accurate calculations
original_image = original_image.astype(float)
reconstructed_image = reconstructed_image.astype(float)

# Calculate PSNR
psnr_value = calculate_psnr(original_image, reconstructed_image)

print(f"PSNR: {psnr_value} dB")


PSNR: 30.716656155643292 dB


In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import random

class CustomDatasetA(Dataset):
    def __init__(self, image_path, patch_size=20):
        self.image = Image.open(image_path).convert("L")  # Convert to grayscale
        self.patch_size = patch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            # transforms.Normalize((0.5,), (0.5,))
        ])

    def __len__(self):
        return 256  # Arbitrary number of samples

    def __getitem__(self, idx):
        # random.seed(idx)
        width, height = self.image.size
        x = random.randint(0, width - self.patch_size)
        y = random.randint(0, height - self.patch_size)
        patch = self.image.crop((x, y, x + self.patch_size, y + self.patch_size))
        # noisy_patch = np.array(patch) + np.random.normal(scale=self.noise_sd, size=(self.patch_size, self.patch_size))
        # noisy_patch = np.clip(noisy_patch, 0, 255).astype(np.uint8)
        # noisy_patch = Image.fromarray(noisy_patch)

        return self.transform(patch)

import random
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset

class CustomDatasetB(Dataset):
    def __init__(self, image_path, patch_size=20):
        self.image = Image.open(image_path).convert("L")  # Convert to grayscale
        self.patch_size = patch_size
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def __len__(self):
        return 256 # Arbitrary number of samples

    def __getitem__(self, idx):
        # random.seed(idx)
        # Get random patch coordinates
        width, height = self.image.size
        x = random.randint(0, width - self.patch_size * 2)
        y = random.randint(0, height - self.patch_size * 2)

        # Extract the patch
        patch = self.image.crop((x, y, x + self.patch_size * 2, y + self.patch_size * 2))

        # Rotate the patch
        angle = random.randint(0, 359)
        rotated_patch = patch.rotate(angle)

        # Get the middle part
        left = (self.patch_size * 2 - self.patch_size) // 2
        top = (self.patch_size * 2 - self.patch_size) // 2
        right = left + self.patch_size
        bottom = top + self.patch_size
        patch = rotated_patch.crop((left, top, right, bottom))
        noisy_patch = np.array(patch)# + np.random.normal(scale=30, size=(self.patch_size, self.patch_size))
        # noisy_patch = np.clip(noisy_patch, 0, 255).astype(np.uint8)
        noisy_patch = Image.fromarray(noisy_patch)
        return self.transform(noisy_patch)

# Usage
image_path = "/content/noisy_lena.jpg"

# Custom dataloaders
loader_a = DataLoader(CustomDatasetA(image_path), batch_size=256, shuffle=True)
loader_b = DataLoader(CustomDatasetB(image_path), batch_size=256, shuffle=True)
import os
import torchvision

# Create directories to save the images
os.makedirs("loader_a_images", exist_ok=True)
os.makedirs("loader_b_images", exist_ok=True)

# Function to save grayscale images from a loader
def save_images(loader, directory):
    for i, batch in enumerate(loader):
        if i >= 3:  # Save only three images
            break
        for j, image in enumerate(batch):
            # Convert to grayscale
            image_gray = torchvision.transforms.functional.to_pil_image(image.squeeze())

            filename = os.path.join(directory, f"image_{i * len(batch) + j}.png")
            image_gray.save(filename)

# Save grayscale images from loader_a
save_images(loader_a, "loader_a_images")

# Save grayscale images from loader_b
save_images(loader_b, "loader_b_images")

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import random

class CustomDatasetA(Dataset):
    def __init__(self, image_path, patch_size=200):
        self.image = Image.open(image_path).convert("L")  # Convert to grayscale
        self.patch_size = patch_size
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def __len__(self):
        return 1 # Arbitrary number of samples

    def __getitem__(self, idx):
        patch = self.image
        noisy_patch = np.array(patch) + np.random.normal(scale=20, size=(self.patch_size, self.patch_size))
        patch = np.clip(noisy_patch, 0, 255).astype(np.uint8)
        noisy_patch = np.array(patch).astype(np.float32) + np.random.normal(scale=20, size=(self.patch_size, self.patch_size)).astype(np.float32)
        noisy_patch = np.clip(noisy_patch, 0, 255).astype(np.uint8)
        noisy_patch = Image.fromarray(noisy_patch)
        return self.transform(noisy_patch)

import random
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset

class CustomDatasetB(Dataset):
    def __init__(self, image_path, patch_size=200):
        self.image = Image.open(image_path).convert("L")  # Convert to grayscale
        self.patch_size = patch_size
        self.transform = transforms.Compose([
            transforms.ToTensor()
        ])

    def __len__(self):
        return 1 # Arbitrary number of samples

    def __getitem__(self, idx):
        patch = self.image
        noisy_patch = np.array(patch) #+ np.random.normal(scale=20, size=(self.patch_size, self.patch_size))
        # noisy_patch = np.clip(noisy_patch, 0, 255).astype(np.uint8)
        # noisy_patch = Image.fromarray(noisy_patch)
        return self.transform(noisy_patch)

# Usage
image_path = "/content/noisy_lena.jpg"
image_path2 = "/content/barbarasmall.jpg"


# Custom dataloaders
loader_a = DataLoader(CustomDatasetB(image_path), batch_size=1, shuffle=True)
loader_b = DataLoader(CustomDatasetB(image_path2), batch_size=1, shuffle=True)
import os
import torchvision

# Create directories to save the images
os.makedirs("loader_a_images", exist_ok=True)
os.makedirs("loader_b_images", exist_ok=True)

# Function to save grayscale images from a loader
def save_images(loader, directory):
    for i, batch in enumerate(loader):
        if i >= 3:  # Save only three images
            break
        for j, image in enumerate(batch):
            # Convert to grayscale
            image_gray = torchvision.transforms.functional.to_pil_image(image.squeeze())

            filename = os.path.join(directory, f"image_{i * len(batch) + j}.png")
            image_gray.save(filename)

# Save grayscale images from loader_a
save_images(loader_a, "loader_a_images")

# Save grayscale images from loader_b
save_images(loader_b, "loader_b_images")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
# Load the Lena image
lena_image = cv2.imread('/content/lena_gray.png', cv2.IMREAD_GRAYSCALE).astype(np.float32)
# print(lena_image)
# Add Gaussian noise with mean 0 and standard deviation 20
noise = (torch.randn_like(torch.tensor(lena_image)) * 20).cpu().numpy()
noisy_image = cv2.add(lena_image, noise)
new_noisy_image = cv2.add(lena_image, noise)
cv2.imwrite('noisy_lena.jpg', noisy_image)

# Define the size of the window
window_size = 7
# Define the averaging kernel of size 5x5
kernel = np.ones((7, 7), np.float32) / 49
# Iterate through the image with a 5x5 window
for i in range(lena_image.shape[0] - window_size + 1):
    for j in range(lena_image.shape[1] - window_size + 1):
        # Extract the current 5x5 window
        # while (window_size >= 3):
          window = new_noisy_image[i:i+window_size, j:j+window_size]

        #   # Check the standard deviation of the window
        #   current_sd = np.std(window)
        #   kernel = np.ones((window_size, window_size), np.float32) / (window_size**2)
        #   # Apply averaging convolution if the standard deviation is between 19 and 21
        #   if 18.8 <= current_sd <= 21.2 and window_size != 3:
          noisy_image[i:i+window_size, j:j+window_size] = cv2.filter2D(window, -1, kernel)
        #       break
        #   elif window_size == 3:
        #       noisy_image[i:i+window_size, j:j+window_size] = cv2.filter2D(window, -1, kernel)
        #   window_size-=1
        # window_size = 15
cv2.imwrite('original_lena.jpg', lena_image)
cv2.imwrite('noisy_lena_a15f.jpg', noisy_image)
# Plot the original, noisy, and denoised images
plt.subplot(131), plt.imshow(lena_image, cmap='gray'), plt.title('Original Lena Image')
plt.subplot(132), plt.imshow(noisy_image, cmap='gray'), plt.title('Noisy Image with Local Averaging')
plt.show()


In [ ]:
# from model import Discriminator, Generator
# from data_loader import get_loaders
# from utils import generate_imgs
from torch import optim
import torch
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn import datasets
from matplotlib import pyplot as plt
import random
# %matplotlib inline

EPOCHS = 300  # 50-300
# BATCH_SIZE = 128
# LOAD_MODEL = False

# IMAGE_SIZE = 32
# A_DS = 'svhn'
A_Channels = 1
# B_DS = 'mnist'
B_Channels = 1

# Directories for storing model and output samples
model_path = './model'
if not os.path.exists(model_path):
    os.makedirs(model_path)
samples_path = './samples'
if not os.path.exists(samples_path):
    os.makedirs(samples_path)
db_path = './data'
if not os.path.exists(samples_path):
    os.makedirs(samples_path)

import torch
import torch.nn as nn
import torch.nn.functional as F


def conv_block(c_in, c_out, k_size=4, stride=2, pad=1, use_bn=True, transpose=False):
    module = []
    if transpose:
        module.append(nn.ConvTranspose2d(c_in, c_out, k_size, stride, pad, output_padding=pad, bias=not use_bn))
    else:
        module.append(nn.Conv2d(c_in, c_out, k_size, stride, pad, bias=not use_bn))
    if use_bn:
        module.append(nn.BatchNorm2d(c_out))
    return nn.Sequential(*module)


class ResBlock(nn.Module):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.conv1 = conv_block(channels, channels, k_size=3, stride=1, pad=1, use_bn=True)
        self.conv2 = conv_block(channels, channels, k_size=3, stride=1, pad=1, use_bn=True)

    def __call__(self, x):
        x = F.relu(self.conv1(x))
        return x + self.conv2(x)


class Discriminator(nn.Module):
    def __init__(self, channels=3, conv_dim=256):
        super(Discriminator, self).__init__()
        self.conv1 = conv_block(channels, conv_dim, use_bn=False)
        self.conv2 = conv_block(conv_dim, conv_dim * 2)
        self.conv3 = conv_block(conv_dim * 2, conv_dim * 4)
        self.conv4 = conv_block(conv_dim * 4, 1, k_size=3, stride=1, pad=1, use_bn=False)

        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.normal_(m.weight, 0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.normal_(m.weight.data, 1.0, 0.02)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        alpha = 0.2
        x = F.leaky_relu(self.conv1(x), alpha)
        x = F.leaky_relu(self.conv2(x), alpha)
        x = F.leaky_relu(self.conv3(x), alpha)
        x = self.conv4(x)
        x = x.reshape([x.shape[0], -1]).mean(1)
        return x


class Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, conv_dim=128):
        super(Generator, self).__init__()
        self.conv1 = conv_block(in_channels, conv_dim, k_size=5, stride=1, pad=2, use_bn=True)
        self.conv2 = conv_block(conv_dim, conv_dim * 2, k_size=3, stride=2, pad=1, use_bn=True)
        self.conv3 = conv_block(conv_dim * 2, conv_dim * 4, k_size=3, stride=2, pad=1, use_bn=True)
        self.res4 = ResBlock(conv_dim * 4)
        self.tconv5 = conv_block(conv_dim * 4, conv_dim * 2, k_size=3, stride=2, pad=1, use_bn=True, transpose=True)
        self.tconv6 = conv_block(conv_dim * 2, conv_dim, k_size=3, stride=2, pad=1, use_bn=True, transpose=True)
        self.conv7 = conv_block(conv_dim, out_channels, k_size=5, stride=1, pad=2, use_bn=False)

        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.normal_(m.weight, 0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.normal_(m.weight.data, 1.0, 0.02)
                nn.init.constant_(m.bias, 0)

    def forward(self, y):
        x = F.relu(self.conv1(y))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.res4(x))
        x = F.relu(self.tconv5(x))
        x = F.relu(self.tconv6(x))
        x = self.conv7(x)
        # noise = torch.randn_like(y) * 20 / 255.0

        # Add noise to the input tensor y
        # noisy_patch =  x+y

        # Clip values to ensure they're within [0, 1]
        # noisy_patch = torch.clamp(noisy_patch, 0, 1)

        return x
class Generator2(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, conv_dim=256):
        super(Generator2, self).__init__()
        self.conv1 = conv_block(in_channels, conv_dim, k_size=5, stride=1, pad=2, use_bn=True)
        self.conv2 = conv_block(conv_dim, conv_dim * 2, k_size=3, stride=2, pad=1, use_bn=True)
        self.conv3 = conv_block(conv_dim * 2, conv_dim * 4, k_size=3, stride=2, pad=1, use_bn=True)
        self.res4 = ResBlock(conv_dim * 4)
        self.tconv5 = conv_block(conv_dim * 4, conv_dim * 2, k_size=3, stride=2, pad=1, use_bn=True, transpose=True)
        self.tconv6 = conv_block(conv_dim * 2, conv_dim, k_size=3, stride=2, pad=1, use_bn=True, transpose=True)
        self.conv7 = conv_block(conv_dim, out_channels, k_size=5, stride=1, pad=2, use_bn=False)

        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.normal_(m.weight, 0.0, 0.02)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.normal_(m.weight.data, 1.0, 0.02)
                nn.init.constant_(m.bias, 0)

    def forward(self, y):
        x = F.relu(self.conv1(y))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.res4(x))
        x = F.relu(self.tconv5(x))
        x = F.relu(self.tconv6(x))
        x = torch.tanh(self.conv7(x))

        return x


In [ ]:
# Networks
ab_gen = Generator2(in_channels=A_Channels, out_channels=B_Channels)
ba_gen = Generator2(in_channels=B_Channels, out_channels=A_Channels)
a_disc = Discriminator(channels=A_Channels)
b_disc = Discriminator(channels=B_Channels)
LOAD_MODEL = False
# Load previous model
if LOAD_MODEL:
    ab_gen.load_state_dict(torch.load(os.path.join(model_path, 'ab_gen.pkl')))
    ba_gen.load_state_dict(torch.load(os.path.join(model_path, 'ba_gen.pkl')))
    a_disc.load_state_dict(torch.load(os.path.join(model_path, 'a_disc.pkl')))
    b_disc.load_state_dict(torch.load(os.path.join(model_path, 'b_disc.pkl')))

# Define Optimizers
g_opt = optim.Adam(list(ab_gen.parameters()) + list(ba_gen.parameters()), lr=0.0002, betas=(0.5, 0.999),
                   weight_decay=2e-5)
d_opt = optim.Adam(list(a_disc.parameters()) + list(b_disc.parameters()), lr=0.0000008, betas=(0.5, 0.999),
                   weight_decay=2e-5)




# a_loader, b_loader = get_loaders(db_path, batch_size=BATCH_SIZE, image_size=IMAGE_SIZE, a_ds=A_DS, b_ds=B_DS)
# iters_per_epoch = min(len(a_loader), len(b_loader))

# # Fix images for viz
# a_fixed = iter(a_loader).next()[0]
# b_fixed = iter(b_loader).next()[0]

# GPU Compatibility
is_cuda = torch.cuda.is_available()
if is_cuda:
    ab_gen, ba_gen = ab_gen.cuda(), ba_gen.cuda()
    a_disc, b_disc = a_disc.cuda(), b_disc.cuda()

    # a_fixed = a_fixed.cuda()
    # b_fixed = b_fixed.cuda()
import torch
import torchvision.transforms.functional as TF

def psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

In [ ]:
import torch
import torchvision.utils as vutils
import math
import os


def generate_imgs(a, b, ab_gen, ba_gen, samples_path, epoch=0):
    ab_gen.eval()
    ba_gen.eval()
    noise = torch.randn_like(a) * 20/255
    a_real = (noise+b)
    # a = (a_real)/torch.max(torch.abs(a_real))-b
    a = torch.clamp(a_real, min=0, max=1)-b
    a = noise
    x=ab_gen(a_real)
    b_fake = (a_real)-x
    a_fake = x

    a_imgs = torch.zeros((a.shape[0] * 2, 3, a.shape[2], a.shape[3]))
    b_imgs = torch.zeros((b.shape[0] * 2, 3, b.shape[2], b.shape[3]))

    even_idx = torch.arange(start=0, end=a.shape[0] * 2, step=2)
    odd_idx = torch.arange(start=1, end=a.shape[0] * 2, step=2)

    a_imgs[even_idx] = a.cpu()
    a_imgs[odd_idx] = b_fake.cpu()

    b_imgs[even_idx] = (b+noise).cpu()
    b_imgs[odd_idx] = a_fake.cpu()

    rows = math.ceil((a.shape[0] * 2) ** 0.5)
    a_imgs_ = vutils.make_grid(a_imgs, normalize=True, nrow=rows)
    b_imgs_ = vutils.make_grid(b_imgs, normalize=True, nrow=rows)

    vutils.save_image(a_imgs_, os.path.join(samples_path, 'a2b_' + str(epoch) + '.png'))
    vutils.save_image(b_imgs_, os.path.join(samples_path, 'b2a_' + str(epoch) + '.png'))

In [ ]:
g_opt2 = optim.Adam(list(ab_gen.parameters()) + list(ba_gen.parameters()), lr=0.00002, betas=(0.5, 0.999),
                   weight_decay=2e-5)
d_opt2 = optim.Adam(list(a_disc.parameters()) + list(b_disc.parameters()), lr=0.000002, betas=(0.5, 0.999),
                   weight_decay=2e-5)

In [ ]:
y = (torch.randn([1, 1, 200, 200])*20).cuda()
y

tensor([[[[  7.9765,  11.1307,  -4.4476,  ...,  32.2226,  -3.9916,   2.8820],
          [ -1.8048, -14.9772,  -3.1419,  ...,   8.8096,   7.5070,  45.3631],
          [ -5.2980,  35.3822, -23.8426,  ...,  16.0243,   2.7240, -17.4197],
          ...,
          [  5.1489,  16.4512, -35.1605,  ..., -29.2908,  34.6418,   2.7005],
          [ 15.3243,   1.1643,  -7.8434,  ...,  -3.7059, -45.8411,  72.1890],
          [ 16.8709,  27.6303,   1.6133,  ..., -44.3636,  27.7413,   5.5693]]]],
       device='cuda:0')

In [ ]:
# from model import Discriminator, Generator
# from data_loader import get_loaders
# from utils import generate_imgs
from torch import optim
import torch
import os



# Data loaders
a_loader, b_loader = loader_a,loader_b
iters_per_epoch = min(len(a_loader), len(b_loader))

# Fix images for viz
a_fixed = next(iter(a_loader))
b_fixed = next(iter(b_loader))

# GPU Compatibility
is_cuda = torch.cuda.is_available()
if is_cuda:
    ab_gen, ba_gen = ab_gen.cuda(), ba_gen.cuda()
    a_disc, b_disc = a_disc.cuda(), b_disc.cuda()

    a_fixed = a_fixed.cuda()
    b_fixed = b_fixed.cuda()

# Cycle-GAN Training
for epoch in range(5000):
    ab_gen.train()
    ba_gen.train()
    # a_disc.train()
    # b_disc.train()

    for i, (a_data, b_data) in enumerate(zip(a_loader, b_loader)):

        # Loading data
        a_real = a_data
        b_real = b_data

        if is_cuda:
            a_real, b_real = a_real.cuda(), b_real.cuda()

        # Fake Images
        # print(a_real.shape)
        # print(b_real.shape)
        a_real = (y/255+b_real)
        a_real = torch.clamp(a_real, min=0, max=1)
        bb=ab_gen(a_real)
        # noise=torch.randn_like(a_real)
        # b_fake = (bb*255 +  noise* 20)
        # max_abs_value = torch.max(torch.abs(b_fake))
        # b_fake = b_fake / max_abs_value
        a_fake = (a_real)-bb
        # # print(ab_gen(a_real))
        noise = torch.randn_like(a_real)*20/255

        # # Training discriminator
        # max_abs_value = torch.max(torch.abs(noise+a_real*255))
        # noise = noise / max_abs_value
        # print(a_fake.shape)
        # print(noise)
        # epoch = 230343
        # break
        b_fake = torch.clamp((bb+ noise), min=0, max=1)
        a_real_out = a_disc(noise)
        a_fake_out = a_disc((a_fake).detach())
        a_d_loss = (torch.mean((a_real_out - 1) ** 2) + torch.mean(a_fake_out ** 2)) / 2
        # print((noise),(a_fake).detach())
        # break
        # # b_real_out = b_disc(a_real)
        # # b_fake_out = b_disc(b_fake.detach())
        # # b_d_loss = (torch.mean((b_real_out - 1) ** 2) + torch.mean(b_fake_out ** 2)) / 2

        d_opt2.zero_grad()
        d_loss = a_d_loss
        d_loss.backward()
        d_opt2.step()

        # Training Generator
        a_fake_out = a_disc(a_fake)
        # b_fake_out = b_disc(b_fake)

        a_g_loss = torch.mean((a_fake_out - 1) ** 2)#
        # b_g_loss = torch.mean((b_fake_out - 1) ** 2)
        g_gan_loss = a_g_loss
        # a_fake_out_2 = a_disc(ba_gen(b_fake))
        # b_fake_out_2 = b_disc(ab_gen(a_fake))
        # a_g_loss2 = torch.mean((a_fake_out_2 - 1) ** 2)
        # b_g_loss2 = torch.mean((b_fake_out_2 - 1) ** 2)
        # u = torch.randn_like(a_real) * 20 / 255.0
        # max_abs_value = torch.max(torch.abs(a_real-a_fake))
        # noise = ((a_real-a_fake))
        # g_gan_loss2 = (noise-bb).abs().mean()
        noise = torch.randn_like(a_real)*20/255
        g_ctnt_loss = (bb - ab_gen(torch.clamp((bb+ noise), min=0, max=1))).abs().mean() #+ ((bb + noise)/torch.max(torch.abs(bb+noise)) - bb - ba_gen(bb + noise)).abs().mean()
        noise = torch.randn_like(a_real)*20/255
        g_ctnt_loss += (bb - ab_gen(torch.clamp((bb+ noise), min=0, max=1))).abs().mean() #+ ((bb + noise)/torch.max(torch.abs(bb+noise)) - bb - ba_gen(bb + noise)).abs().mean()
        # g_ctnt_loss = a_g_ctnt_loss + b_g_ctnt_loss
        for v in range(20):
          noise = torch.randn_like(a_real)*20/255
          g_ctnt_loss += (bb - ab_gen(torch.clamp((bb+ noise), min=0, max=1))).abs().mean() #+ ((bb + noise)/torch.max(torch.abs(bb+noise)) - bb - ba_gen(bb + noise)).abs().mean()
        # x=(b_fake - a_real).abs().mean()
        g_opt2.zero_grad()
        g_loss =  g_gan_loss+(g_ctnt_loss/22)
        g_loss.backward()
        g_opt2.step()

        if i % 50 == 0:
            print("Epoch: " + str(epoch + 1) + "/" + str(EPOCHS)
                  + " it: " + str(i) + "/" + str(iters_per_epoch)
                  + "\ta_d_loss:" + str(round(a_d_loss.item(), 4))
                  + "\ta_g_loss:" + str(round(a_g_loss.item(), 4))
                  + "\tg_ctnt_loss:" + str(round(g_ctnt_loss.item(), 4))
                  + "\tb_d_loss:" + str(round(psnr(b_real, bb.detach()).item(), 4))
                  + "\tb_g_loss:" + str(round(x.item(), 4))
                  + "\tb_g_ctnt_loss:" + str(round(g_gan_loss.item(), 4)))

    torch.save(ab_gen.state_dict(), os.path.join(model_path, 'ab_gen.pkl'))
    torch.save(ba_gen.state_dict(), os.path.join(model_path, 'ba_gen.pkl'))
    torch.save(a_disc.state_dict(), os.path.join(model_path, 'a_disc.pkl'))
    torch.save(b_disc.state_dict(), os.path.join(model_path, 'b_disc.pkl'))
    if epoch%10==0:
      generate_imgs(a_fixed, b_fixed, ab_gen, ba_gen, samples_path, epoch=epoch + 1)

generate_imgs(a_fixed, b_fixed, ab_gen, ba_gen, samples_path)

Epoch: 1/300 it: 0/1	a_d_loss:0.2734	a_g_loss:0.2283	g_ctnt_loss:2.2012	b_d_loss:17.6267	b_g_loss:0.0779	b_g_ctnt_loss:0.2283
Epoch: 2/300 it: 0/1	a_d_loss:0.2725	a_g_loss:0.229	g_ctnt_loss:2.1317	b_d_loss:17.65	b_g_loss:0.0779	b_g_ctnt_loss:0.229
Epoch: 3/300 it: 0/1	a_d_loss:0.2723	a_g_loss:0.2301	g_ctnt_loss:2.0372	b_d_loss:17.6663	b_g_loss:0.0779	b_g_ctnt_loss:0.2301
Epoch: 4/300 it: 0/1	a_d_loss:0.2713	a_g_loss:0.2308	g_ctnt_loss:1.9769	b_d_loss:17.7141	b_g_loss:0.0779	b_g_ctnt_loss:0.2308
Epoch: 5/300 it: 0/1	a_d_loss:0.2714	a_g_loss:0.2303	g_ctnt_loss:1.9577	b_d_loss:17.7824	b_g_loss:0.0779	b_g_ctnt_loss:0.2303
Epoch: 6/300 it: 0/1	a_d_loss:0.2716	a_g_loss:0.2302	g_ctnt_loss:1.9362	b_d_loss:17.8381	b_g_loss:0.0779	b_g_ctnt_loss:0.2302
Epoch: 7/300 it: 0/1	a_d_loss:0.2721	a_g_loss:0.2301	g_ctnt_loss:1.913	b_d_loss:17.8838	b_g_loss:0.0779	b_g_ctnt_loss:0.2301
Epoch: 8/300 it: 0/1	a_d_loss:0.2719	a_g_loss:0.23	g_ctnt_loss:1.902	b_d_loss:17.9272	b_g_loss:0.0779	b_g_ctnt_loss:0.23
Ep

In [ ]:
rm -rf samplesloader_b_images

In [ ]:
import torch
import torchvision.utils as vutils

# Function to generate images using the model
def generate_images(model, dataloader, num_images):
    device = next(model.parameters()).device
    model.eval()
    image_accumulator = torch.zeros(3, 200, 200).to(device)  # Assuming output shape is (3, 200, 200)
    total_batches = 0

    with torch.no_grad():
        for batch_idx, data in enumerate(dataloader):
            images = data.to(device)
            outputs = model(images)
            image_accumulator += torch.sum(outputs, dim=0)  # Accumulate images
            total_batches += 1

            if total_batches * dataloader.batch_size >= num_images:
                break

    average_image = image_accumulator / (total_batches * dataloader.batch_size)
    return average_image

# Assuming you have a PyTorch dataloader named 'dataloader'
# with images of shape (batch_size, channels, height, width)
# and a model named 'model'

num_images = 65536*2*2*2
batch_size = 32

# Generate images using the dataloader
average_image = generate_images(ab_gen, loader_a, num_images)

# Save the average image
vutils.save_image(average_image, 'average_image.png')


In [ ]:
!git clone https://github.com/s-chh/Pytorch-CycleGAN-Digits.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python /content/Pytorch-CycleGAN-Digits/main.py

In [ ]:
rm -rf /content/samples

In [ ]:
!python /content/Pytorch-CycleGAN-Digits/test.py

In [ ]:
!python /content/Pytorch-CycleGAN-Digits/test.py

In [ ]:
!python /content/Pytorch-CycleGAN-Digits/test.py

In [ ]:
import shutil
import os

# Folder path you want to download
folder_path = '/content/model/a20'

# Zip the folder
shutil.make_archive("/content/examples2", 'zip', folder_path)


In [ ]:
!python /content/Pytorch-CycleGAN-Digits/summary.py